In [ ]:
#!pip install -q ipython-autotime
%load_ext autotime
%load_ext autoreload
%timeit
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import torch 
from torch import nn
from Utils import split_data, TextDataset, Training
from models import ANN, CNN1D, BILSTM
from torch.utils.data import Dataset, DataLoader



In [ ]:
loaded_data = np.load("cleaned-dataset\data_npy.npy")
X_train, X_test, y_train, y_test = split_data(loaded_data)

training_dataset = TextDataset(X_train, y_train)
testing_dataset = TextDataset(X_test, y_test)
train_loader = DataLoader(dataset=training_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(dataset=training_dataset, batch_size=128, shuffle=True)

In [ ]:
def acc_func(y_true, y_pred):
    y_pred = torch.round(y_pred)
    return (y_true == y_pred).sum() / len(y_true)

In [ ]:
x, y = next(iter(train_loader))
x.shape, x.unsqueeze(1).shape, x.unsqueeze(-1).shape, x.unsqueeze(0).shape

In [39]:
INPUT_SIZE = x.shape[1]



100

time: 140 ms (started: 2023-07-24 20:08:48 +03:00)


In [ ]:
#* GLOABL VARIABLES
EPOCHS = 150
LEARNING_RATE = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# DEVICE = torch.device("cpu")
# model = LSTMModel().to(DEVICE)

#* LSTM 
INPUT_SIZE = x.shape[1]
HIDDEN_STATE = 64
NUM_LAYERS = 4
NUM_CLASSES = 1 #* binary classification

model = BILSTM(INPUT_SIZE, HIDDEN_STATE, NUM_LAYERS, NUM_CLASSES, bidirection=True).to(DEVICE)


loss = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)




In [ ]:
# torch.save(ann.state_dict(), "model.pth")
# ann_loaded = ANN().to(DEVICE)
# ann_loaded.load_state_dict(torch.load("model.pth"))

In [ ]:
history = Training(
    model,
    train_loader,
    test_loader,
    EPOCHS,
    DEVICE,
    loss,
    optimizer,
    print_every=1,
)

In [ ]:
# pool of size=3, stride=2
m = nn.MaxPool1d(3, stride=2)
input = torch.randn(100, 1, 1)
output = m(input)
output.shape

In [ ]:
# Importing metrics
from sklearn.metrics import accuracy_score,confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


# Prediction
y_pred = ann(torch.tensor(X_test).float().to(DEVICE)).to("cpu").detach().numpy().reshape(-1)


# Accuracy score
print("Model accuracy is ",accuracy_score(y_pred,y_test))

confusion_matrix = confusion_matrix(y_pred=y_pred,y_true=y_test)

fig,ax = plt.subplots(figsize=(6,6))
sns.heatmap(confusion_matrix,annot=True,fmt="0.1f",linewidths=1.5)
plt.show()